In [3]:
import joblib 
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from experiments_helpers import ExperimentResults

if 'src' in os.getcwd():
    os.chdir('..')
    print(os.getcwd())

folder = 'results_bench'
name = 'torch-gs-mlp-b7'

idx = 0
es_base: ExperimentResults = ExperimentResults.load_results_from_file(f'{folder}/{name}_{idx}.joblib')

es_base.pretty_print_robust_vs_base()

ExperimentResults with 18 metrics and 3 artifacts.
############################## Metrics ##############################
------------------------- Base metrics -------------------------
validity: 1.00 (std: 0.00)
proximityL1: 0.63 (std: 0.48)
lof: -1.27 (std: 0.40)
cf_counterfactual_stability: 0.03 (std: 0.08)
generation_time: 0.01 (std: 0.00)
------------------------- Base metrics 2 -------------------------
validity_2: 0.12 (std: 0.33)
proximityL1_2: 0.63 (std: 0.48)
lof_2: -1.27 (std: 0.40)
cf_counterfactual_stability_2: 0.43 (std: 0.19)
------------------------- Robust metrics -------------------------
robust_validity: 1.00 (std: 0.00)
robust_proximityL1: 2.73 (std: 1.80)
robust_lof: -1.78 (std: 0.61)
robust_cf_counterfactual_stability: 0.31 (std: 0.26)
robust_generation_time: 64.37 (std: 75.48)
------------------------- Robust metrics 2 -------------------------
robust_validity_2: 0.50 (std: 0.50)
robust_proximityL1_2: 2.73 (std: 1.80)
robust_lof_2: -1.78 (std: 0.61)
robust_cf_cou